In [23]:
! pip install streamlit langchain langchain-openai langchain_experimental langchain_experimental langgraph

  Using cached langchain_experimental-0.0.58-py3-none-any.whl (199 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 2.0 MB/s eta 0:00:00


In [25]:
import streamlit as st
import os
from langchain.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_agent_executor
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
import operator

## Define Environment Variable

In [ ]:
langchain_api_key = '<api key>'
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] ="multi-agent"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<azure openai endpoint>"
os.environ["AZURE_OPENAI_API_KEY"] = "<azure openai api key>"

In [ ]:
llm = AzureChatOpenAI(temperature=0,
                          max_tokens=1024,openai_api_version="2023-07-01-preview",
    azure_deployment="gpt-4-1106-preview")

## Graph Node - Programmer

#### Programmer Agent - Specialized in Writing Program based on Requirements

In [ ]:
class Code(BaseModel):
    """Plan to follow in future"""

    code: str = Field(
        description="Detailed optmized error-free Python code on the provided requirements"
    )


from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

code_gen_prompt = ChatPromptTemplate.from_template(
    '''**Role**: You are a expert software python programmer. You need to develop python code
**Task**: As a programmer, you are required to complete the function. Use a Chain-of-Thought approach to break
down the problem, create pseudocode, and then write the code in Python language. Ensure that your code is
efficient, readable, and well-commented.

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the task.
2. **Algorithm/Method Selection**: Decide on the most efficient way.
3. **Pseudocode Creation**: Write down the steps you will follow in pseudocode.
4. **Code Generation**: Translate your pseudocode into executable Python code

*REQURIEMENT*
{requirement}'''
)
coder = create_structured_output_runnable(
    Code, llm, code_gen_prompt
)

In [ ]:
code_ = coder.invoke({'requirement':'Generate fibbinaco series'})

## Graph Node - Tester

#### Tester Agent - Generates input test cases and expected output

In [ ]:
class Test(BaseModel):
    """Plan to follow in future"""

    Input: List[List] = Field(
        description="Input for Test cases to evaluate the provided code"
    )
    Output: List[List] = Field(
        description="Expected Output for Test cases to evaluate the provided code"
    )


from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

test_gen_prompt = ChatPromptTemplate.from_template(
    '''**Role**: As a tester, your task is to create Basic and Simple test cases based on provided Requirement and Python Code.
These test cases should encompass Basic, Edge scenarios to ensure the code's robustness, reliability, and scalability.
**1. Basic Test Cases**:
- **Objective**: Basic and Small scale test cases to validate basic functioning
**2. Edge Test Cases**:
- **Objective**: To evaluate the function's behavior under extreme or unusual conditions.
**Instructions**:
- Implement a comprehensive set of test cases based on requirements.
- Pay special attention to edge cases as they often reveal hidden bugs.
- Only Generate Basics and Edge cases which are small
- Avoid generating Large scale and Medium scale test case. Focus only small, basic test-cases
*REQURIEMENT*
{requirement}
**Code**
{code}
'''
)
tester_agent = create_structured_output_runnable(
    Test, llm, test_gen_prompt
)

In [ ]:
print(code_.code)

# Chain of Thought
# 1. Understand and Clarify: The task is to generate the Fibonacci series.
# 2. Algorithm/Method Selection: Use an iterative approach to generate the series efficiently.
# 3. Pseudocode Creation:
#    - Initialize the first two numbers of the series: 0 and 1.
#    - Use a loop to generate the next numbers in the series.
#    - For each iteration, calculate the next number by summing the last two numbers.
#    - Update the last two numbers with the new values.
# 4. Code Generation:

# Function to generate Fibonacci series up to n terms

def generate_fibonacci(n):
    # Initialize the first two numbers
    a, b = 0, 1
    series = []

    # Generate the series up to n terms
    for _ in range(n):
        series.append(a)
        # Update the last two numbers
        a, b = b, a + b

    return series

# Example usage:
# Generate the first 10 terms of the Fibonacci series
print(generate_fibonacci(10))


In [ ]:
test_ = tester_agent.invoke({'requirement':'Generate fibbinaco series','code':code_.code})

In [ ]:
test_

Test(Input=[[0], [1], [2], [5], [10], [-1], [1.5], ['three'], [None]], Output=[[[]], [[0]], [[0, 1]], [[0, 1, 1, 2, 3]], [[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]], [[]], [[]], [[]], [[]]])

## Graph Node - Python Executor

#### Executor - Executes code in a Python environment on provided test cases

In [ ]:
class ExecutableCode(BaseModel):
    """Plan to follow in future"""

    code: str = Field(
        description="Detailed optmized error-free Python code with test cases assertion"
    )

python_execution_gen = ChatPromptTemplate.from_template(
    """You have to add testing layer in the *Python Code* that can help to execute the code. You need to pass only Provided Input as argument and validate if the Given Expected Output is matched.
*Instruction*:
- Make sure to return the error if the assertion fails
- Generate the code that can be execute
Python Code to excecute:
*Python Code*:{code}
Input and Output For Code:
*Input*:{input}
*Expected Output*:{output}"""
)
execution = create_structured_output_runnable(
    ExecutableCode, llm, python_execution_gen
)

In [ ]:
code_execute = execution.invoke({"code":code_.code,"input":test_.Input,'output':test_.Output})

In [ ]:
print(code_execute.code)

# Chain of Thought
# 1. Understand and Clarify: The task is to generate the Fibonacci series.
# 2. Algorithm/Method Selection: Use an iterative approach to generate the series efficiently.
# 3. Pseudocode Creation:
#    - Initialize the first two numbers of the series: 0 and 1.
#    - Use a loop to generate the next numbers in the series.
#    - For each iteration, calculate the next number by summing the last two numbers.
#    - Update the last two numbers with the new values.
# 4. Code Generation:

# Function to generate Fibonacci series up to n terms

def generate_fibonacci(n):
    # Initialize the first two numbers
    a, b = 0, 1
    series = []

    # Generate the series up to n terms
    for _ in range(n):
        series.append(a)
        # Update the last two numbers
        a, b = b, a + b

    return series

# Testing layer

def test_generate_fibonacci():
    test_cases = [
        ([0], []),
        ([1], [0]),
        ([2], [0, 1]),
        ([5], [0, 1, 1, 2, 3]),
        (

In [ ]:
error = None
try:
    exec(code_execute.code)
except Exception as e:
    error = f'Exception : {e}'
error

"Exception : 'float' object cannot be interpreted as an integer"

## Graph Node -Debugger

#### Debugger - Debugs code using LLM knowledge and sends it back to the 'Executer' Agent in case of errors.

In [ ]:
class RefineCode(BaseModel):

    code: str = Field(
        description="Optimized and Refined Python code to resolve the error"
    )


python_refine_gen = ChatPromptTemplate.from_template(
    """You are expert in Python Debugging. You have to analysis Given Code and Error and generate code that handles the error
    *Instructions*:
    - Make sure to generate error free code
    - Generated code is able to handle the error

    *Code*: {code}
    *Error*: {error}
    """
)
refine_code = create_structured_output_runnable(
    RefineCode, llm, python_refine_gen
)

In [ ]:
dummy_json = {
    "code": code_execute.code,
    "error": error
}

In [ ]:
refine_code_ = refine_code.invoke(dummy_json)

In [ ]:
print(refine_code_.code)

# Function to generate Fibonacci series up to n terms

def generate_fibonacci(n):
    # Check if n is an integer and greater than 0
    if not isinstance(n, int) or n <= 0:
        return []

    # Initialize the first two numbers
    a, b = 0, 1
    series = []

    # Generate the series up to n terms
    for _ in range(n):
        series.append(a)
        # Update the last two numbers
        a, b = b, a + b

    return series

# Testing layer

def test_generate_fibonacci():
    test_cases = [
        (0, []),
        (1, [0]),
        (2, [0, 1]),
        (5, [0, 1, 1, 2, 3]),
        (10, [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]),
        (-1, []),
        (1.5, []),
        ('three', []),
        (None, [])
    ]

    for inputs, expected in test_cases:
        result = generate_fibonacci(inputs)
        assert result == expected, f'Error on test case {inputs}, expected {expected}, got {result}'

    print("All test cases passed!")

test_generate_fibonacci()


In [ ]:
exec(refine_code_.code)

All test cases passed!


## Graph Design

In [ ]:
class AgentCoder(TypedDict):
    requirement: str
    code: str
    tests: Dict[str, any]
    errors: Optional[str]

In [ ]:
def programmer(state):
    print(f'Entering in Programmer')
    requirement = state['requirement']
    code_ = coder.invoke({'requirement':requirement})
    return {'code':code_.code}

def debugger(state):
    print(f'Entering in Debugger')
    errors = state['errors']
    code = state['code']
    refine_code_ = refine_code.invoke({'code':code,'error':errors})
    return {'code':refine_code_.code,'errors':None}

def executer(state):
    print(f'Entering in Executer')
    tests = state['tests']
    input_ = tests['input']
    output_ = tests['output']
    code = state['code']
    executable_code = execution.invoke({"code":code,"input":input_,'output':output_})
    #print(f"Executable Code - {executable_code.code}")
    error = None
    try:
        exec(executable_code.code)
        print("Code Execution Successful")
    except Exception as e:
        print('Found Error While Running')
        error = f"Execution Error : {e}"
    return {'code':executable_code.code,'errors':error}

def tester(state):
    print(f'Entering in Tester')
    requirement = state['requirement']
    code = state['code']
    tests = tester_agent.invoke({'requirement':requirement,'code':code})
    #tester.invoke({'requirement':'Generate fibbinaco series','code':code_.code})
    return {'tests':{'input':tests.Input,'output':tests.Output}}

def decide_to_end(state):
    print(f'Entering in Decide to End')
    if state['errors']:
        return 'debugger'
    else:
        return 'end'

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentCoder)

# Define the nodes
workflow.add_node("programmer", programmer)
workflow.add_node("debugger", debugger)
workflow.add_node("executer", executer)
workflow.add_node("tester", tester)
#workflow.add_node('decide_to_end',decide_to_end)

# Build graph
workflow.set_entry_point("programmer")
workflow.add_edge("programmer", "tester")
workflow.add_edge("debugger", "executer")
workflow.add_edge("tester", "executer")
#workflow.add_edge("executer", "decide_to_end")

workflow.add_conditional_edges(
    "executer",
    decide_to_end,
    {
        "end": END,
        "debugger": "debugger",
    },
)

# Compile
app = workflow.compile()

- Running Leetcode Problem (https://leetcode.com/problems/two-sum/description/)
- It should generate test-cases to evaluate the solution
- The solution should be optimized

In [ ]:
requirement = """Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target. You may assume that each input would have exactly one solution, and you may not use the same element twice.You can return the answer in any order."""

In [ ]:
from langchain_core.messages import HumanMessage

config = {"recursion_limit": 50}
inputs = {"requirement": requirement}
running_dict = {}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        running_dict[k] = v
        if k != "__end__":
            print(v)
            print('----------'*20)

Entering in Programmer
{'code': '# Understand and Clarify:\n# The task is to find two numbers in the array that add up to the target value.\n# Each input is guaranteed to have one solution, and we cannot use the same element twice.\n\n# Algorithm/Method Selection:\n# A hash map (dictionary in Python) can be used for efficient lookup of the complement of each element.\n# We iterate through the array, and for each element, we check if the complement (target - element) is in the map.\n# If it is, we return the current index and the index stored in the map.\n# If not, we add the element and its index to the map.\n\n# Pseudocode Creation:\n# 1. Initialize an empty dictionary to store elements and their indices.\n# 2. Iterate through the array with both index and value.\n# 3. Calculate the complement by subtracting the current value from the target.\n# 4. Check if the complement is in the dictionary.\n#    - If yes, return the current index and the index from the dictionary.\n#    - If no, a

In [ ]:
print(v['requirement'])

Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target. You may assume that each input would have exactly one solution, and you may not use the same element twice.You can return the answer in any order.


In [ ]:
print(v['code'])

def two_sum(nums, target):
    num_dict = {}
    for index, num in enumerate(nums):
        complement = target - num
        if complement in num_dict:
            return [num_dict[complement], index]
        num_dict[num] = index

# Testing layer
if __name__ == '__main__':
    test_cases = [
        ([[2, 7, 11, 15], 9], [0, 1]),
        ([[3, 2, 4], 6], [1, 2]),
        ([[0, 4, 3, 0], 0], [0, 3]),
        ([[-1, -2, -3, -4, -5], -8], [2, 4])
    ]

    for inputs, expected in test_cases:
        result = two_sum(*inputs)
        assert result == expected, f'Test failed for input {inputs}: expected {expected}, got {result}'
        print(f'Test passed for input {inputs}: expected {expected}, got {result}')



In [ ]:
running_dict

{'programmer': {'code': '# Understand and Clarify:\n# The task is to find two numbers in the array that add up to the target value.\n# Each input is guaranteed to have one solution, and we cannot use the same element twice.\n\n# Algorithm/Method Selection:\n# A hash map (dictionary in Python) can be used for efficient lookup of the complement of each element.\n# We iterate through the array, and for each element, we check if the complement (target - element) is in the map.\n# If it is, we return the current index and the index stored in the map.\n# If not, we add the element and its index to the map.\n\n# Pseudocode Creation:\n# 1. Initialize an empty dictionary to store elements and their indices.\n# 2. Iterate through the array with both index and value.\n# 3. Calculate the complement by subtracting the current value from the target.\n# 4. Check if the complement is in the dictionary.\n#    - If yes, return the current index and the index from the dictionary.\n#    - If no, add the c